In [2]:
#feature selection for knnregressor 
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split, cross_val_score
import matplotlib.pylab as plt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
plt.style.use('ggplot')

In [41]:
train = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')

#train = train.loc[train['y'] <= 200]

train['set'] = 1
test['set']  = 0
data  = pd.concat([train, test])
test_id = test['ID']

data.reset_index(drop=True, inplace=True)

bin_features = data.columns[data.dtypes == int]
cat_features = data.columns[data.dtypes == object]

duplicate_features = ['X205', 'X385', 'X60', 'X44', 'X157', 'X67', 'X39', 'X263', 'X279',
       'X130', 'X299', 'X254', 'X226', 'X326', 'X360', 'X199', 'X262',
       'X214', 'X239', 'X53', 'X71', 'X84', 'X134', 'X147', 'X222', 'X48',
       'X216', 'X62', 'X35', 'X37', 'X119', 'X293', 'X330', 'X76', 'X146',
       'X296', 'X58', 'X364', 'X227', 'X382']

low_std_feat = [ 'X11', 'X93', 'X107',  'X233', 'X235', 'X268', 'X289', 'X290', 'X297', 'X347' ]
low_inique_feat = ['X4']

data.drop(duplicate_features, 1, inplace=True)

In [42]:
cat_features = data.columns[data.dtypes == object]
bin_features    = data.columns[data.dtypes == int ]
bin_features = bin_features.drop(['ID','set'])

In [43]:
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))
        data[c] = lbl.transform(list(data[c].values))
data.head()

,ID,X0,X1,X10,X100,X101,X102,X103,X104,X105,...,X92,X93,X94,X95,X96,X97,X98,X99,set,y
0,0,37,23,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,130.81
1,6,37,21,0,1,1,0,0,0,0,...,0,0,0,0,1,0,1,0,1,88.53
2,7,24,24,0,0,1,0,0,0,0,...,0,0,0,0,1,0,1,0,1,76.26
3,9,24,21,0,0,1,0,0,0,0,...,0,0,0,0,1,0,1,0,1,80.62
4,13,24,23,0,0,1,0,0,0,0,...,0,0,0,0,1,0,1,0,1,78.02


In [44]:
x_train = data.loc[data['set'] == 1].drop(['set','ID','y'], 1)
x_test  = data.loc[data['set'] == 0].drop(['set','ID','y'], 1)
y_train = data.loc[data['set'] == 1, 'y'].values
y_mean = y_train.mean()

print x_train.shape
print y_train.shape

(4209, 336)
(4209,)


In [45]:
%%time
estimator = KNeighborsRegressor(n_neighbors = 100, weights = 'uniform', metric='minkowski', p = 2, n_jobs = -1 , 
                          algorithm='auto')

cv_score = cross_val_score(estimator, x_train, y_train, scoring='r2', cv = 5, n_jobs=-1)

print cv_score.mean()
print cv_score

0.230552546781
[ 0.30107165  0.1332402   0.26531821  0.2484496   0.20468307]
CPU times: user 364 ms, sys: 240 ms, total: 604 ms
Wall time: 3.86 s


In [49]:
# number of epochs
epochs = 50
#initial propability
p0 = 0.1
#crossingover probability
p_cross = 0.5
#mutate probability
p_mut = 0.01
#number of the best samples
N_best = 33
N_features = x_train.shape[1]
#initialization 
samples = np.random.binomial(1, p0, ( N_best, N_features)).astype(bool)

all_features = x_train.columns
scores = np.array([])

for epoch in range(epochs):
    for i in range(N_best):
        j = np.random.randint(0, N_best)
        cross = np.random.binomial(1, p_cross, N_features).astype(bool)
        if (samples[i][cross] != samples[j][cross]).any():
            new_sample = np.copy(samples[i])
            new_sample[cross] = np.copy(samples[j][cross])
            if new_sample.any():
                samples = np.vstack([samples, new_sample])
    for i in range(N_best, len(samples)):
        mutate = np.random.binomial(1, p_mut, N_features).astype(bool)
        samples[i][mutate] = ~samples[i][mutate]
    for i in range(int(N_best*0.2)):
        new_sample = np.random.binomial(1, p0, N_features).astype(bool)
        samples = np.vstack([samples, new_sample])
    for i, sample in enumerate(samples[N_best:]):
        score = cross_val_score(estimator, x_train[all_features[sample]] , y_train, scoring = 'r2', cv=5, ).mean()
        scores = np.append(scores, score)
    ind_best = scores.argsort()[::-1]
    print 'epoch = ', epoch+1
    print scores[ind_best][:5]
    scores = scores[ind_best][:N_best]
    samples = samples[ind_best]
    samples = samples[:N_best]

epoch =  1
[ 0.50734658  0.50209809  0.48149059  0.46136259  0.45702716]
epoch =  2
[ 0.50734658  0.50209809  0.49392462  0.48149059  0.46136259]
epoch =  3
[ 0.50734658  0.50209809  0.49392462  0.49164035  0.49056116]
epoch =  4
[ 0.50734658  0.50311492  0.50209809  0.49392462  0.49169031]
epoch =  5
[ 0.50734658  0.50645789  0.50412124  0.50311492  0.50209809]
epoch =  6
[ 0.50734658  0.50645789  0.50630276  0.50627316  0.50412124]
epoch =  7
[ 0.50970438  0.50771718  0.50734658  0.50645789  0.50630276]
epoch =  8
[ 0.52637507  0.50970438  0.50935043  0.50775098  0.50771718]
epoch =  9
[ 0.52637507  0.50970438  0.50967449  0.50935043  0.5093396 ]
epoch =  10
[ 0.52637507  0.50974418  0.50970438  0.50967449  0.50942863]
epoch =  11
[ 0.52894806  0.52637507  0.51172546  0.51152571  0.51144392]
epoch =  12
[ 0.52894806  0.52637507  0.51943669  0.51231729  0.51174037]
epoch =  13
[ 0.52894806  0.52800639  0.52637507  0.51943669  0.51349572]
epoch =  14
[ 0.52894806  0.52826842  0.5280063

In [52]:
np.array(all_features[samples[0]].tolist())

array(['X118', 'X127', 'X139', 'X152', 'X159', 'X16', 'X173', 'X174',
       'X184', 'X19', 'X215', 'X228', 'X229', 'X232', 'X234', 'X238',
       'X249', 'X252', 'X255', 'X256', 'X260', 'X261', 'X267', 'X268',
       'X275', 'X276', 'X277', 'X278', 'X282', 'X288', 'X289', 'X311',
       'X313', 'X315', 'X316', 'X32', 'X320', 'X337', 'X339', 'X345',
       'X346', 'X357', 'X365', 'X372', 'X379', 'X38', 'X52', 'X59', 'X64',
       'X65', 'X80', 'X82', 'X89', 'X98'], 
      dtype='|S4')

In [53]:
knn_features = ['X118', 'X127', 'X139', 'X152', 'X159', 'X16', 'X173', 'X174',
       'X184', 'X19', 'X215', 'X228', 'X229', 'X232', 'X234', 'X238',
       'X249', 'X252', 'X255', 'X256', 'X260', 'X261', 'X267', 'X268',
       'X275', 'X276', 'X277', 'X278', 'X282', 'X288', 'X289', 'X311',
       'X313', 'X315', 'X316', 'X32', 'X320', 'X337', 'X339', 'X345',
       'X346', 'X357', 'X365', 'X372', 'X379', 'X38', 'X52', 'X59', 'X64',
       'X65', 'X80', 'X82', 'X89', 'X98']